In [ ]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
from gcspy import GraphOfConvexSets

In [ ]:
n = 20
radius = .1
q_init = np.zeros(2)
q_term = np.ones(2)
z_max = 1
vel = 1
alpha = 5
beta = 1
autonomy = z_max / alpha * vel

In [ ]:
C = []
r = []
np.random.seed(1)
while len(C) < n:
    ci = np.random.rand(2)
    ri = np.random.rand() * radius
    keep = True
    for cj, rj in zip(C, r):
        dij = np.linalg.norm(ci - cj)
        if dij < ri + rj:
            keep = False
            break
    if keep:
        C.append(ci)
        r.append(ri)
C = np.array(C)
r = np.array(r)

In [ ]:
def plot_islands():
    plt.gca().add_patch(plt.Rectangle((-radius, -radius), 1 + 2 * radius, 1 + 2 * radius, fc='azure'))
    for i in range(n):
        plt.gca().add_patch(plt.Circle(C[i], r[i],
                                       ec='k', fc='lightgreen'))
    plt.gca().set_aspect('equal')
    plt.xticks([])
    plt.yticks([])
    plt.xlim([-radius, 1 + radius])
    plt.ylim([-radius, 1 + radius])

In [ ]:
gcs = GraphOfConvexSets()

s = gcs.add_vertex("s")
qs = s.add_variable(2)
s.add_constraint(qs == 0)
zs = s.add_variable(1)
s.add_constraint(zs == z_max)

t = gcs.add_vertex("t")
qt = t.add_variable(2)
t.add_constraint(qt == 1)
zt = t.add_variable(1)
t.add_constraint(zt >= 0)
t.add_constraint(zt <= z_max)

for i in range(n):
    vi = gcs.add_vertex(f"v{i}")
    qi = vi.add_variable(2)
    vi.add_constraint(cp.norm(qi - C[i], 2) <= r[i])
    zi0 = vi.add_variable(1)
    vi.add_constraint(zi0 >= 0)
    vi.add_constraint(zi0 <= z_max)
    zi1 = vi.add_variable(1)
    vi.add_constraint(zi1 >= 0)
    vi.add_constraint(zi1 <= z_max)
    ti = vi.add_variable(1)
    vi.add_constraint(ti >= 0)
    vi.add_constraint(ti <= z_max / beta)
    vi.add_cost(ti)
    vi.add_constraint(zi1 == zi0 + ti * beta)

In [ ]:
for i in range(n):
    vi = gcs.get_vertex_by_name(f"v{i}")
    qi, zi0, zi1, ti = vi.variables
    
    ds = np.linalg.norm(C[i] - q_init)
    if ds < autonomy + r[i]:
        qs, zs = s.variables
        e = gcs.add_edge(s, vi)
        tsi = cp.norm(qi - qs, 2) / vel
        e.add_cost(tsi)
        e.add_constraint(zi0 <= zs - alpha * tsi)
        
    dt = np.linalg.norm(C[i] - q_term)
    if dt < autonomy + r[i]:
        qt, zt = t.variables
        e = gcs.add_edge(vi, t)
        tti = cp.norm(qi - qt, 2) / vel
        e.add_cost(tti)
        e.add_constraint(zt <= zi1 - alpha * tti)
        
    for j in range(n):
        if i != j:
            dij = np.linalg.norm(C[i] - C[j])
            if dij < autonomy + r[i] + r[j]:
                vj = gcs.get_vertex_by_name(f"v{j}")
                qj, zj0, zj1, tj = vj.variables
                e = gcs.add_edge(vi, vj)
                tij = cp.norm(qi - qj, 2) / vel
                e.add_cost(tij)
                e.add_constraint(zj0 <= zi1 - alpha * tij)

In [ ]:
gcs.graphviz()

In [ ]:
prob = gcs.solve_shortest_path(s, t)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
path = [s]
path_edges = []
while path[-1] != t:
    for e in gcs.outgoing_edges(path[-1]):
        if e.y.value is not None and e.y.value > 1e-4:
            path.append(e.head)
            path_edges.append(e)
            break

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 5))

gcs.plot_subgraph_2d()

plot_islands()
plt.savefig('spp_drone_flight.pdf', bbox_inches='tight')

In [ ]:
battery_levels = [s.variables[1].value[0]]
times = [0]
q = s.variables[0].value
for v in path[1:-1]:
    
    battery_levels.append(v.variables[1].value[0])
    q_next = v.variables[0].value
    time_v = np.linalg.norm(q_next - q) / vel
    times.append(times[-1] + time_v)
    q = q_next
    
    battery_levels.append(v.variables[2].value[0])
    times.append(times[-1] + v.variables[3].value[0])
    
battery_levels.append(t.variables[1].value[0])
time_t = np.linalg.norm(t.variables[0].value[0] - q) / vel
times.append(times[-1] + time_t)

battery_levels  = np.array(battery_levels) * 100
times = np.array(times)

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot([times[0], times[-1]], [100, 100], 'r--')
plt.plot([times[0], times[-1]], [0, 0], 'r--')
plt.plot(times, battery_levels)
plt.xlabel('time')
plt.ylabel('battery level (%)')
plt.xlim([times[0], times[-1]])
plt.grid()
plt.savefig('spp_drone_battery.pdf', bbox_inches='tight')